In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import LSTM,Dense,Dropout,Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [ ]:
data = pd.read_csv("/content/spam.csv",encoding='latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
data.iloc[0].v2

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [ ]:
data.info()

In [ ]:
data['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [ ]:
x = data['v2']
y = data['v1']
le = LabelEncoder()
y = le.fit_transform(y)
y = y.reshape(-1,1)  ## convert y to 2d array (not mandatory)
y

array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=0.85,random_state=7)

In [ ]:
xtrain[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [ ]:
tok = Tokenizer(num_words=1000)
tok.fit_on_texts(xtrain)
sequences = tok.texts_to_sequences(xtrain)
sequences_mat = pad_sequences(sequences,maxlen=150,padding="pre")
sequences_mat

array([[  0,   0,   0, ...,   0, 155, 447],
       [  0,   0,   0, ...,   2,  28, 113],
       [  0,   0,   0, ...,  38,  44,   4],
       ...,
       [  0,   0,   0, ...,  52,   2,  28],
       [  0,   0,   0, ..., 193, 249,  97],
       [  0,   0,   0, ...,   2,  31, 531]], dtype=int32)

In [ ]:
sequences[0]

[155, 447]

In [ ]:
sequences_mat[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0, 155, 447], dtype=int32)

In [ ]:
sequences_mat.shape

(4736, 150)

In [ ]:
tok.word_index

In [ ]:
from keras.models import Sequential
model = Sequential()
model.add(Embedding(1000,10,input_length=150))
model.add(LSTM(64))
model.add(Dense(256,activation='relu'))  ## hidden layer
model.add(Dropout(0.5))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
model.fit(sequences_mat,ytrain,batch_size=64,epochs=15)

Epoch 1/15
74/74 [==============================] - 12s 111ms/step - loss: 0.3774 - accuracy: 0.8682
Epoch 2/15
74/74 [==============================] - 7s 93ms/step - loss: 0.1050 - accuracy: 0.9734
Epoch 3/15
74/74 [==============================] - 8s 107ms/step - loss: 0.0540 - accuracy: 0.9850
Epoch 4/15
74/74 [==============================] - 7s 94ms/step - loss: 0.0420 - accuracy: 0.9878
Epoch 5/15
74/74 [==============================] - 8s 108ms/step - loss: 0.0338 - accuracy: 0.9909
Epoch 6/15
74/74 [==============================] - 7s 100ms/step - loss: 0.0324 - accuracy: 0.9913
Epoch 7/15
74/74 [==============================] - 8s 101ms/step - loss: 0.0269 - accuracy: 0.9930
Epoch 8/15
74/74 [==============================] - 8s 109ms/step - loss: 0.0235 - accuracy: 0.9937
Epoch 9/15
74/74 [==============================] - 8s 109ms/step - loss: 0.0202 - accuracy: 0.9937
Epoch 10/15
74/74 [==============================] - 8s 109ms/step - loss: 0.0188 - accuracy: 0.9956


In [ ]:
text_sequences = tok.texts_to_sequences(xtest)
test_sequences_mat = pad_sequences(text_sequences,maxlen=150,padding="pre")

In [ ]:
model.evaluate(test_sequences_mat,ytest)

27/27 [==============================] - 1s 25ms/step - loss: 0.0636 - accuracy: 0.9868


[0.063572458922863, 0.9868420958518982]

In [ ]:
ypred = model.predict(test_sequences_mat)
ypred = ypred.round()
ypred

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(ytest,ypred)
ac = accuracy_score(ytest,ypred)
print(ac)
print(cm)

0.9868421052631579
[[719   4]
 [  7 106]]


In [ ]:
### Test a new sample
txt = ["You got a 50% discount and free of cost hampers in every purchase",
       "Congratulations, you are selected for the applied job"]
txt = tok.texts_to_sequences(txt)
txt = pad_sequences(txt,maxlen=150,padding="pre")
model.predict(txt).round()

1/1 [==============================] - 0s 57ms/step


array([[1.],
       [0.]], dtype=float32)